In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("UNSW_NB15_training-set.csv")
df.head()

In [ ]:
df.info()

In [ ]:
categoric = [col for col in df.columns
             if df[col].dtype == 'object']
print("Categorical columns :", categoric)

numeric = [col for col in df.columns
            if df[col].dtype != 'object']
print("Numerical columns: ",numeric)

In [ ]:
df = df.drop(columns=['proto', 'service', 'state', 'attack_cat'])

In [ ]:
# Normal veriden 2000 örnek, attackten 500 örnek 
normal = df[df["label"] == 0].sample(2000, random_state=42)
attack = df[df["label"] == 1].sample(500, random_state=42)

df_sample = pd.concat([normal,attack],ignore_index=True)

df_sample.shape

In [ ]:
df_sample.head()

In [ ]:
df_sample.info()

In [ ]:
df_sample.isnull().sum()

In [ ]:
from sklearn.preprocessing import StandardScaler

X = df_sample.drop(["label"], axis=1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import confusion_matrix, classification_report, recall_score, precision_score, f1_score

In [ ]:
dbscan = DBSCAN(eps=2.2, min_samples=5)
labels = dbscan.fit_predict(X_scaled)

df_sample["DBSCAN_Cluster"] = labels

In [ ]:
# DBSCAN için eps parametresini belirlemek için k-nearest neighbors analizi
neighbors = NearestNeighbors(n_neighbors=5)
neighbors_fit = neighbors.fit(X_scaled)
distances, indices = neighbors_fit.kneighbors(X_scaled)

# En uzak 5. komşuya olan mesafeleri sırala
distances = np.sort(distances[:, 4])
plt.plot(distances)
plt.title('DBSCAN için eps tahmini (dirsek noktası)')
plt.show()

# Dirsek noktasını belirlemek için
knee_point = np.argmax(np.diff(distances))
print(f"Önerilen eps değeri: {distances[knee_point]:.3f}")

In [ ]:
# eps tahmini çizgisi ile birlikte k-distance plot
plt.figure(figsize=(8, 5))
plt.plot(distances)
plt.axhline(y=2.2, color='r', linestyle='--', label='eps ≈ 2.2')
plt.title("DBSCAN için eps tahmini (dirsek noktası)")
plt.xlabel("Veri Noktası Sırası")
plt.ylabel("5. Komşuya Uzaklık")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(8,6))
sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=labels, palette="tab10", s=30)
plt.title("DBSCAN Kümeleme - PCA 2D")
plt.xlabel("PCA1")
plt.ylabel("PCA2")
plt.legend(title="Cluster")
plt.show()


In [ ]:
# Küme etiketleri içinde -1 olanlar 'gürültü' yani anomali
print(df_sample["DBSCAN_Cluster"].value_counts())

# Gürültü içinde kaç attack var?
anomaliler = df_sample[df_sample["DBSCAN_Cluster"] == -1]
print("Gürültüdeki Attack Sayısı:", anomaliler["label"].sum())

In [ ]:
df_sample["Predicted_Attack"] = (df_sample["DBSCAN_Cluster"]== -1).astype(int)

In [ ]:
df_sample.head()

In [ ]:
print(confusion_matrix(df_sample["label"], df_sample["Predicted_Attack"]))
print(classification_report(df_sample["label"], df_sample["Predicted_Attack"]))

In [ ]:
from sklearn.metrics import silhouette_score

eps_values = np.arange(1.5, 3.0, 0.1)
scores = []

for eps in eps_values:
    db = DBSCAN(eps=eps, min_samples=5)
    labels = db.fit_predict(X_scaled)
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

    # En az 2 küme varsa skoru hesapla (outlier olabilir, sorun değil)
    if n_clusters >= 2:
        try:
            score = silhouette_score(X_scaled, labels)
            scores.append((eps, score))
        except:
            continue  # silhouette score hesaplanamazsa geç

if scores:
    best_eps = max(scores, key=lambda x: x[1])[0]
    print(f"En iyi eps değeri: {best_eps}")
else:
    best_eps = 2.2



In [ ]:
print(df["label"].value_counts())

In [ ]:
normal = df[df["label"] == 0].sample(2000, random_state=42)
attack = df[df["label"] == 1].sample(500, random_state=42)

df_sample = pd.concat([normal,attack],ignore_index=True)
X = df_sample.drop(["label"], axis=1) 
y = df_sample["label"]

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# DBSCAN'ı tekrar çalıştır
dbscan = DBSCAN(eps=2.2, min_samples=5)
labels = dbscan.fit_predict(X_scaled)

# Predicted_Attack sütununu oluştur
df_sample["DBSCAN_Cluster"] = labels
df_sample["Predicted_Attack"] = (df_sample["DBSCAN_Cluster"] == -1).astype(int)

In [ ]:
iso = IsolationForest(contamination=len(attack)/len(df_sample), random_state=42)
df_sample["Predicted"] = iso.fit_predict(X_scaled)

In [ ]:
df_sample["Predicted"] = df_sample["Predicted"].apply(lambda x: 1 if x == -1 else 0)

In [ ]:
print(confusion_matrix(y, df_sample["Predicted"]))
print(classification_report(y, df_sample["Predicted"]))

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=df_sample["Predicted"], palette="tab10", s=30)
plt.title("Isolation Forest Anomali Tespiti - PCA 2D")
plt.xlabel("PCA1")
plt.ylabel("PCA2")
plt.legend(title="Anomali")
plt.show()

In [ ]:
print("=== MODEL KARŞILAŞTIRMASI ===")
print("\nDBSCAN Sonuçları:")
print(f"Precision: {precision_score(y, df_sample['Predicted_Attack']):.3f}")
print(f"Recall: {recall_score(y, df_sample['Predicted_Attack']):.3f}")
print(f"F1-Score: {f1_score(y, df_sample['Predicted_Attack']):.3f}")

print("\nIsolation Forest Sonuçları:")
print(f"Precision: {precision_score(y, df_sample['Predicted']):.3f}")
print(f"Recall: {recall_score(y, df_sample['Predicted']):.3f}")
print(f"F1-Score: {f1_score(y, df_sample['Predicted']):.3f}")

In [ ]:
print("\n=== SONUÇ YORUMLAMA ===")
print("DBSCAN:")
print("- Saldırı tespiti precision: 0.296 (düşük)")
print("- Saldırı tespiti recall: 0.116 (çok düşük)")
print("- Normal trafik tespiti: 0.81 (iyi)")

print("\nIsolation Forest:")
print("- Saldırı tespiti precision: 0.258 (düşük)")
print("- Saldırı tespiti recall: 0.258 (daha iyi)")
print("- Normal trafik tespiti: 0.81 (iyi)")


---

Bu projede, **UNSW-NB15** siber güvenlik veri setini kullanarak denetimsiz öğrenme yöntemleriyle ağ saldırılarını anomali olarak tespit etmeyi amaçladık. Gerçek etiketleri kullanmadan, sadece veri yapısına dayalı olarak saldırıları tespit etmeye çalıştık.

## Amaç
- **DBSCAN** ve **Isolation Forest** algoritmalarını karşılaştırmak
- Hangi yöntemin siber saldırı tespitinde daha etkili olduğunu belirlemek
- Denetimsiz öğrenmenin anomali tespitindeki rolünü değerlendirmek


### Model Performans Karşılaştırması

| Metrik | DBSCAN | Isolation Forest |
|--------|--------|------------------|
| **Precision** | 0.296 | 0.258 |
| **Recall** | 0.116 | 0.258 |
| **F1-Score** | 0.167 | 0.258 |

**DBSCAN:**
-  Daha yüksek precision (0.296)
-  Çok düşük recall (0.116)
-  Normal trafik tespitinde başarılı (0.81)

**Isolation Forest:**
-  Daha dengeli precision/recall (0.258/0.258)
-  Daha yüksek recall (0.258)
-  Normal trafik tespitinde başarılı (0.81)

**DBSCAN Kullanın Eğer:**
- Düşük false positive oranı istiyorsanız
- Precision önemliyse
- Kümeleme tabanlı yaklaşım tercih ediyorsanız

**Isolation Forest Kullanın Eğer:**
- Daha fazla saldırı tespit etmek istiyorsanız
- Recall önemliyse
- Anomali tabanlı yaklaşım tercih ediyorsanız

1. **Her iki model de** normal trafiği iyi tespit ediyor (0.81 precision)
2. **Isolation Forest** saldırı tespitinde daha dengeli performans gösteriyor
3. **DBSCAN** daha seçici, daha az ama daha doğru tespit yapıyor
4. **Denetimsiz öğrenme** siber güvenlikte etkili bir yaklaşım sunuyor

---
